# Trabalho Prático 1

- O tabalho pode ser feito em duplas ou individualmente
- A realização da entrega deverá ser feita via Teams (tarefa adicionada à equipe), atente-se ao prazo de entrega. Não será possível realizar a entrega após o prazo previsto.
- Apenas esse arquivo (.ipynb) com a resolução deverá ser entregue (entregas em formato .zip serão penalizados)
- Caso tenha sido feito em dupla, apenas 1 aluno deve realizar a entrega, com os nomes dos dois participantes

Amanda Silva Guimarães / GEC / P7 / L5
Pedro

Considere um tabuleiro quadrado onde cada bloco pode estar LIMPO "o" ou SUJO "i", seu trabalho é mover o aspirador "x" pelo tabuleiro a fim de limpar todos os blocos sujos.
- Sempre que o aspirador deixa um bloco, esse pode ser considerado como limpo.
- O objetivo é deixar todos espaços limpos (tabuleiro preenchido apenas com 'o' e com o aspirador 'x')
- Para responder às questões a, b, c, d, e basta completar os códigos 

<img src="tabuleiro.jpg" style=width:300px;height:200px/>




## Classe do problema

#### a) Implemente a função Heurística: h(n) = nº de espaços limpos "o" (Maximizar o número de estados limpos)
#### (O restante da classe já está pronto e não precisa ser alterado)

In [12]:
import numpy as np
class Aspirador():
    def __init__(self, tamanho):
        '''
        Construtor
        Args:
            - tamanho: quantidade de linhas e colunas
        '''
        self.tamanho = tamanho
        
    def encontra_posicao(self, estado, elemento):
        '''
        Varre todo o tabuleiro (estado) e verifica em qual posição 'elemento' está
        Args:
            - estado: matriz contendo o estado do tabuleiro
            - elemento: elemento a ser buscado na matriz
        Return:
            - Retorna a linha e coluna onde o elemento se encontra
        '''
        for i in range(self.tamanho):
            for j in range(self.tamanho):
                if estado[i, j] == elemento:
                    return i, j
        return None, None
    
    def verifica_objetivo(self, estado):
        '''
        Verifica se o estado atual é o objetivo
        Objetivo: Não haver sujeira ("i") -> Todos blocos, exceto onde o aspirador está
        devem ser "o"
        Args:
            - estado: estado atual do tabuleiro
        Return:
            - booleano dizendo se o estado atual é ou não o objetivo
        '''
        item, cont = np.unique(estado, return_counts = True)
        mapa = dict()
        for i in range(len(item)):
            mapa[item[i]] = cont[i]
            
        # Todos elementos exceto onde o aspirador está
        if mapa['o'] == (self.tamanho**2 - 1):
            return True
        
        return False
    
    def expande_estados(self, atual):
        '''
        Dado o estado atual, realiza a expansão de estados
        Args:
            - atual: matriz que descreve o estado atual
        Return:
            - lista com os novos estados após a expansão
        '''
        
        novos_estados = []
        linha, coluna = self.encontra_posicao(atual, 'x')

        # Cima
        if linha > 0:
            novo_estado = np.copy(atual)
            nova_linha = linha - 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Baixo
        if linha < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_linha = linha + 1

            novo_estado[nova_linha, coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)


        # Esquerda
        if coluna > 0:
            novo_estado = np.copy(atual)
            nova_coluna = coluna - 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        # Direita
        if coluna < self.tamanho - 1:
            novo_estado = np.copy(atual)
            nova_coluna = coluna + 1

            novo_estado[linha, nova_coluna] = 'x'
            novo_estado[linha, coluna] = 'o'

            novos_estados.append(novo_estado)

        return novos_estados
        
    def heuristica(self, atual, objetivo):
        '''
        Calcula a distância de Manhattan entre o estado atual e o estado objetivo
        Args: 
            - atual: estado atual do tabuleiro
            - objetivo: estado objetivo do problema
        Return:
            - Distância de Manhattan
        '''
        distancia_total = 0
        
        for i in range(self.problema):
            for j in range(self.problema):
                
                bloco_atual = atual[i, j]
                linha, coluna = self._encontra_posicao(objetivo, bloco_atual)
                
                distancia = abs(linha - i) + abs(coluna - j)
                distancia_total += distancia

        return distancia_total
    

#### b) Implemente a busca gulosa utilizando a heurística criada no item a.
#### Dica: Lembrar que a função heapq ordena do menor para o maior, mas agora queremos maximizar a heurística, logo devemos usar um comando da própria heapq para ordená-la de forma decrescente 

In [13]:
import heapq

class BuscaGulosa():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca gulosa, armazenando os estados em uma FILA DE PRIORIDADES
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        
        '''
        OBS.: A distância de Manhattan é inversamente proporcional à prioridade, quanto menor a distância, maior
        a prioridade desse estado
        '''
        
        p_fila = []
        
        # Heuristica, ID, elemento
        id_estado = 0
        heapq.heappush(p_fila, (0, id_estado, inicio))

        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0

        while not len(p_fila) == 0:
            atual = heapq.heappop(p_fila)[2]
            estados_visitados.append(atual)
            print(f"Estado:\n{atual}")

            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
            else:
                cont_estados += 1
                print("Visitando #", cont_estados)
                novos_estados = self.problema.expande_estados(atual)
                print("Estados Gerados:")
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        id_estado += 1
                        distancia = self.problema.heuristica(i, fim)
                        print(i)
                        print(distancia)
                        heapq.heappush(p_fila, (distancia, id_estado, i))

        return solucao_encontrada, estados_visitados, cont_estados

#### c) Implementar a função de busca em profundidade

In [14]:
from queue import LifoQueue

class DepthFirstSearch():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca DFS, armazenando os estados em uma PILHA
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        piha = LifoQueue()
        piha.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not piha.empty():
            atual = piha.get()
            estados_visitados.append(atual)
            
            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                print(f"Visitando #{cont_estados}")
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        print(i)
                        piha.put(i)

        return solucao_encontrada, estados_visitados, cont_estados

#### d) Implementar a função de busca em largura

In [15]:
from queue import Queue

class BreadthFirstSearch():
    def __init__(self, problema):
        '''
        Construtor
        Args:
            - problema: objeto do problema a ser solucionado
        '''
        self.problema = problema
        
    def compara_estados(self, estado, estado_visitado):
        '''
        Comparar dois estados, caso haja alguma diferença, é retornado Falso, caso sejam idênticos é retornado True
        Args:
            - estado: estado atual
            - estado_visitado: estado para fazer a comparação com o estado atual
            
        Return:
            - Retorna se os estados são iguais ou não
        '''
        for i in range(self.problema.tamanho):
            for j in range(self.problema.tamanho):
                if estado[i, j] != estado_visitado[i, j]:
                    return False
        return True
    
    def verifica_visitados(self, estado, estados_visitados):
        '''
        Verificar se um estado está na lista de visitados
        Args:
            - estado: estado atual
            - estados_visitados: lista com todos os estados visitados
        '''
        for estado_i in estados_visitados:
            if self.compara_estados(estado, estado_i):
                return True
            
        return False
    
    
    def busca(self, inicio, fim):
        '''
        Realiza a busca BFS, armazenando os estados em uma FILA
        Args:
            - inicio: estado inicial do problema
            - fim: estado objetivo
        Return:
            - booleano se a solução foi encontrada, lista dos estados visitados, quantidade de estados visitados
        '''
        fila = Queue()
        fila.put(inicio)
        
        solucao_encontrada = False
        estados_visitados = []
        cont_estados = 0
        
        while not fila.empty():
            atual = fila.get()
            estados_visitados.append(atual)
            
            if self.problema.verifica_estados(atual, fim):
                solucao_encontrada = True
                break
                
            else:
                cont_estados += 1
                print(f"Visitando #{cont_estados}")
                novos_estados = self.problema.expande_estados(atual)
                for i in novos_estados:
                    if not self._verifica_visitado(i, estados_visitados):
                        print(i)
                        fila.put(i)
                        
        return solucao_encontrada, estados_visitados, cont_estados

#### e) Execute as 3 buscas com o tabuleiro inicial fornecido e compare: a quantidade de estados buscados (inclusive o estado objetivo) e se a solução foi encontrada ou não.

In [16]:
# Criando objeto do problema
problema = Aspirador(3)

# Criando Matriz inicial
start = np.array([['i','o','o'],['i','x','o'],['o','i','i']])
start

array([['i', 'o', 'o'],
       ['i', 'x', 'o'],
       ['o', 'i', 'i']], dtype='<U1')

In [17]:
#Execução da busca gulosa
bg = BuscaGulosa(problema)

bg_solucao, bg_estados_visitados, bg_num_visitados = bg.busca(start, target) # chamando busca

if bg_solucao:
    print("Solução encontrada com ", bg_num_visitados, " passos")
else:
    print("Solução não encontrada!")


NameError: name 'target' is not defined

In [18]:
#Execução da BFS
bfs = BreadthFirstSearch(problema)

bfs_solucao, bfs_estados_visitados, bfs_num_visitados = bfs.busca(start, target) # chamando busca

if bfs_solucao:
    print("Solução encontrada")
else:
    print("Solução não encontrada!")


NameError: name 'target' is not defined

In [19]:
#Execução da DFS
dfs = DepthFirstSearch(problema)

dfs_solucao, dfs_estados_visitados, dfs_num_visitados = dfs.busca(start, target) # chamando busca

if dfs_solucao:
    print("Solução encontrada")
else:
    print("Solução não encontrada!")


NameError: name 'target' is not defined